<a href="https://colab.research.google.com/github/fjadidi2001/Cyber-Attack-Detection/blob/main/DL4cyber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Setup Environment and Unzip the Dataset

- Install necessary libraries.
- Mount Google Drive and unzip the dataset.

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import os
import zipfile

In [3]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Path to the ZIP file in Google Drive
zip_path = '/content/drive/MyDrive/network-intrusion-dataset.zip'
extract_dir = '/content/cic-ids2017/'

In [5]:
# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [6]:
# List extracted files
csv_files = [f for f in os.listdir(extract_dir) if f.endswith('.csv')]
print("Extracted CSV files:", csv_files)

Extracted CSV files: ['Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', 'Friday-WorkingHours-Morning.pcap_ISCX.csv', 'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', 'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', 'Monday-WorkingHours.pcap_ISCX.csv', 'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', 'Wednesday-workingHours.pcap_ISCX.csv', 'Tuesday-WorkingHours.pcap_ISCX.csv']


In [7]:
# Visualization: Table of Extracted Files
files_df = pd.DataFrame(csv_files, columns=['CSV Files'])
print("\nTable of Extracted Files:")
print(files_df)


Table of Extracted Files:
                                           CSV Files
0   Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
1          Friday-WorkingHours-Morning.pcap_ISCX.csv
2  Friday-WorkingHours-Afternoon-PortScan.pcap_IS...
3  Thursday-WorkingHours-Afternoon-Infilteration....
4                  Monday-WorkingHours.pcap_ISCX.csv
5  Thursday-WorkingHours-Morning-WebAttacks.pcap_...
6               Wednesday-workingHours.pcap_ISCX.csv
7                 Tuesday-WorkingHours.pcap_ISCX.csv


# Step 2: Load and Preprocess Data

- Combine all CSV files into a single DataFrame.
- Handle missing values, encode categorical features, and normalize numerical features.
- Split into training, validation, and test sets.
- Visualize class distribution and feature correlations.

In [8]:
# Combine all CSV files into one DataFrame
dataframes = []
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(extract_dir, csv_file))
    dataframes.append(df)

In [9]:
data = pd.concat(dataframes, ignore_index=True)

In [10]:
# Clean column names (remove leading/trailing spaces)
data.columns = data.columns.str.strip()

In [12]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [11]:
# Handle missing values with median imputation
imputer = SimpleImputer(strategy='median')
data_imputed = pd.DataFrame(imputer.fit_transform(data.select_dtypes(include=[np.number])),
                            columns=data.select_dtypes(include=[np.number]).columns)
for col in data.select_dtypes(exclude=[np.number]).columns:
    data_imputed[col] = data[col].fillna(data[col].mode()[0])  # Mode for categorical

ValueError: Input X contains infinity or a value too large for dtype('float64').

# Step 3: Build and Train the Deep Learning Model

- Use a Multi-Layer Perceptron (MLP) for classification.
- Train with validation and plot training history.

# Step 4: Evaluate and Visualize Results

- Compute metrics (accuracy, precision, recall, F1-score).
- Generate confusion matrix, ROC curve, pie chart of predictions, and scatter plots.

In [ ]:
# Step 1: Setup Environment and Unzip Dataset
!pip install tensorflow seaborn



# Path to the ZIP file in Google Drive (adjust the path as needed)
zip_path = '/content/drive/MyDrive/network-intrusion-dataset.zip'
extract_dir = '/content/cic-ids2017/'

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List extracted files
csv_files = [f for f in os.listdir(extract_dir) if f.endswith('.csv')]
print("Extracted CSV files:", csv_files)

# Step 2: Load and Preprocess Data
# Combine all CSV files into one DataFrame
dataframes = []
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(extract_dir, csv_file))
    dataframes.append(df)

data = pd.concat(dataframes, ignore_index=True)

# Clean column names (remove leading/trailing spaces)
data.columns = data.columns.str.strip()

# Handle missing values (drop rows with NaN or infinite values)
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

# Encode categorical features (e.g., 'Label')
le = LabelEncoder()
data['Label'] = le.fit_transform(data['Label'])

# Separate features and labels
X = data.drop('Label', axis=1)
y = data['Label']

# Normalize numerical features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Visualization 1: Class Distribution
plt.figure(figsize=(10, 6))
sns.countplot(x=data['Label'])
plt.title('Class Distribution in CIC-IDS2017')
plt.xlabel('Class (Encoded)')
plt.ylabel('Count')
plt.show()

# Visualization 2: Correlation Heatmap (using a subset for speed)
subset = data.sample(1000, random_state=42)
plt.figure(figsize=(12, 10))
sns.heatmap(subset.corr(), annot=False, cmap='coolwarm')
plt.title('Feature Correlation Heatmap')
plt.show()

# Step 3: Build and Train the Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y_train)), activation='softmax')  # Multi-class output
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

# Visualization 3: Training History
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss Over Epochs')
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy Over Epochs')
plt.legend()
plt.show()

# Step 4: Evaluate and Visualize Results
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

# Predict on test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# Classification Report
print("Classification Report:\n", classification_report(y_test, y_pred_classes))

# ROC Curve (for binary classification; adapt for multi-class if needed)
if len(np.unique(y_test)) == 2:  # Binary case
    fpr, tpr, _ = roc_curve(y_test, y_pred[:, 1])
    roc_auc = auc(fpr, tpr)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title('ROC Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.show()
else:
    print("ROC curve not plotted (multi-class scenario).")

# Visualization 4: Pie Chart of Predicted Classes
pred_counts = pd.Series(y_pred_classes).value_counts()
plt.figure(figsize=(6, 6))
plt.pie(pred_counts, labels=pred_counts.index, autopct='%1.1f%%')
plt.title('Predicted Class Distribution')
plt.show()

# Visualization 5: Scatter Plot of Two Features (e.g., first two features)
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_test[:, 0], y=X_test[:, 1], hue=y_pred_classes, palette='deep')
plt.title('Scatter Plot of First Two Features by Predicted Class')
plt.xlabel('Feature 1 (Normalized)')
plt.ylabel('Feature 2 (Normalized)')
plt.show()